In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from scipy.stats import linregress
# def autocorrelation(x):
#     """
#     Compute autocorrelation of a time series x.
#     """
#     n = len(x)
#     variance = np.var(x)
#     x = x - np.mean(x)
#     autocorr_f = np.correlate(x, x, mode='full')[-n:]
#     autocorr_f /= (variance * np.arange(n, 0, -1))
#     return autocorr_f

# Example file reading (replace with actual file)
# Assuming the Cassandra output file contains data in a CSV format
# with a column for the quantity of interest, say 'Energy'
#33fa0c17dcc62689206c71f315a2bef8/prod.out.box1.prp")) #57a003793cdd64f6b86d4472fe31f7da
df_box1 = np.genfromtxt("/scratch365/mcarlozo/generalizedFF/opt_ff_ms/workspace/c9a306c8b7a1cab8fea0a0fcccf8d682/gemc.eq.out.box2.prp")
df_box1 = np.genfromtxt("/scratch365/mcarlozo/generalizedFF/opt_ff_ms/workspace/7abef0781e0e1b311fbff7bed1d8cf43/gemc.eq.out.box2.prp")
# df_box1 = np.genfromtxt("/scratch365/mcarlozo/generalizedFF/opt_ff_ms/workspace/6f3da4aec70e16d462786d82b8a77afe/prod.out.box2.prp")
# df_box1 = np.genfromtxt("/scratch365/mcarlozo/generalizedFF/opt_ff_ms/workspace/514489e76022f8180cfb413d911db92f/prod.out.box2.prp")
# df_box1 = np.genfromtxt("/scratch365/mcarlozo/generalizedFF/opt_ff_ms/workspace/38ad1aa8eb2011f713a9a22a09da9f3b/prod.out.box2.prp")
density_col = 6
pressure_col = 3
enrg_col = 2
n_mols_col = 5
# pull property and take average
energy = df_box1[:, 2 - 1]

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statistics import linear_regression


# Generate some sample data
data = energy

# Step 1: Standardize the data
def is_slope_zero(n, simulations_per_block, threshold=0.03):
    # Step 1: Divide the Data
    n_length = len(n)
    # print(f"Total number of data points: {n_length}")

    # Calculate the number of blocks
    m = n_length // simulations_per_block
    # print(f"Number of blocks: {m}")

    # Initialize averages list
    averages = []

    for i in range(m):
        start_index = i * simulations_per_block
        # Handle the last block to include all remaining elements
        end_index = n_length if i == m - 1 else (i + 1) * simulations_per_block
        # print(f"Block {i + 1}: Start index = {start_index}, End index = {end_index}")

        # Calculate the average of the current block
        section_average = np.mean(n[start_index:end_index])
        averages.append(section_average)

    # Step 2: Scale the Averages between 0 and 1
    scaler = StandardScaler()  # You can change to MinMaxScaler() if desired
    scaled_averages = scaler.fit_transform(np.array(averages).reshape(-1, 1)).flatten()
    

    # Step 3: Divide into 5 groups
    group_size = m // 5
    slopes = []
    intercepts = []
    
    for j in range(5):
        group_start = j * group_size
        group_end = (j + 1) * group_size if j < 4 else m  # Last group gets any remaining blocks
        
        if group_end - group_start < 2:  # Need at least two points to calculate slope
            print(f"Group {j + 1} has insufficient data points.")
            continue
        
        x_group = np.arange(group_start, group_end)
        y_group = scaled_averages[group_start:group_end]
        slope, intercept, r_value, p_value, std_err = linregress(x_group, y_group)
        slopes.append(slope)
        intercepts.append(intercept)
        print(f"Group {j + 1}: Slope = {slope}, Intercept = {intercept}")

    # Step 3: Fit a Line with intercept fixed at 0
    x = np.arange(m)
    slope, intercept, r_value, p_value, std_err = linregress(x, scaled_averages)
    
    # if abs(slope) > threshold:
    # Plotting
    plt.figure(figsize=(10, 6))

    # Original Data
    plt.subplot(1, 2, 1)
    plt.plot(scaler.transform(n.reshape(-1, 1)), label='Original Data', color='blue')
    
    # Draw vertical lines for each block average
    for i in range(m):
        # Calculate the x position scaled based on total number of data
        x_position = (i * simulations_per_block + (simulations_per_block / 2))  # Middle of the block
        plt.axvline(x=x_position, color='orange', linestyle='--', lw=0.7,
                    label=f'Block {i + 1} Average' if i == 0 else "")
    

    plt.title('Original Data')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()

    # Averages and Line of Best Fit
    plt.subplot(1, 2, 2)
    plt.scatter(x, scaled_averages, label='Block Averages', color='orange')
    
    # Plot the line of best fit with intercept fixed at 0
    plt.plot(x, slope * x + intercept, label='Line of Best Fit (Intercept = 0)', color='red')
    plt.title('Block Averages and Line of Best Fit')
    plt.xlabel('Block')
    plt.ylabel('Scaled Average')
    plt.axhline(y=0, color='k', linestyle='--', lw=0.7)  # y=0 line for reference
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Step 4: Check the Slope
    return slope, abs(slope) < threshold
slope, result = is_slope_zero(energy, 10)
print(f"The slope of the data {slope} is approximately zero: {result}")